## Entrenamiento para usar python 

In [1]:
def convertToBase13(num):
    if num == 0:
        return "0"

    # Digits for base 13
    base13_digits = "0123456789ABC"  
    digits = ""
    positive = abs(num)
    
    while positive > 0:
        # Append digits/letters
        digits += base13_digits[positive % 13]  
        positive = positive // 13

    # Reverse the string at the end
    reversed_digits = digits[::-1]  
    if num < 0:
        return "-" + reversed_digits
    else:
        return reversed_digits

In [5]:
num = int(input("Escriba un numero"))
convertToBase13(num)

'A'

In [2]:
def func(x):
    if x == 1:
        def rv():
            print("X is equal to 1")
    else:
        def rv():
            print("X is not 1")

    return rv

In [5]:
new_func = func(2)
new_func

<function __main__.func.<locals>.rv()>

In [6]:
import inspect
from queue import Queue

print(inspect.getsource(Queue))

class Queue:
    '''Create a queue object with a given maximum size.

    If maxsize is <= 0, the queue size is infinite.
    '''

    def __init__(self, maxsize=0):
        self.maxsize = maxsize
        self._init(maxsize)

        # mutex must be held whenever the queue is mutating.  All methods
        # that acquire mutex must release it before returning.  mutex
        # is shared between the three conditions, so acquiring and
        # releasing the conditions also acquires and releases mutex.
        self.mutex = threading.Lock()

        # Notify not_empty whenever an item is added to the queue; a
        # thread waiting to get is notified then.
        self.not_empty = threading.Condition(self.mutex)

        # Notify not_full whenever an item is removed from the queue;
        # a thread waiting to put is notified then.
        self.not_full = threading.Condition(self.mutex)

        # Notify all_tasks_done whenever the number of unfinished tasks
        # drops to zero; th

# Validador de Agendamiento de Reuniones (Calendar Booking System)

## 📝 Descripción:
Construirás un sistema de agendamiento de reuniones que:

    - Permita a usuarios crear reuniones con fecha, hora de inicio y duración.

    - Valide que no haya solapamientos entre reuniones existentes.

    - Permita consultar las reuniones existentes de un día.

    - Permita cancelar reuniones.

## 🎯 Requisitos funcionales:
    
    - Crear reunión (schedule_meeting)

    - Parámetros: start_time, duration, description

    - Valida que no se cruce con ninguna otra existente.

    - Listar reuniones (list_meetings(date))

    - Devuelve lista ordenada por hora.

    - Cancelar reunión (cancel_meeting(id))

## 💡 Consideraciones técnicas:

    - Usa datetime para manejar fechas y horas.

    - Las reuniones duran una cantidad de minutos (duration).

    - Valida que una reunión no se cruce en el tiempo con otra.

In [1]:
from datetime import datetime, timedelta
from typing import List
import uuid

class Meeting:
    def __init__(self, start_time: datetime, duration: int, description: str):
        self.id = str(uuid.uuid4())
        self.start_time = start_time
        self.end_time = start_time + timedelta(minutes=duration)
        self.duration = duration
        self.description = description

    def __str__(self):
        return f"{self.description} ({self.start_time.strftime('%H:%M')} - {self.end_time.strftime('%H:%M')})"

class MeetingScheduler:
    def __init__(self):
        self.meetings: List[Meeting] = []

    def schedule_meeting(self, start: str, duration: int, description: str):
        start_dt = datetime.strptime(start, "%Y-%m-%d %H:%M")
        new_meeting = Meeting(start_dt, duration, description)

        for meeting in self.meetings:
            if self._overlaps(meeting, new_meeting):
                raise ValueError("La nueva reunión se cruza con una existente.")

        self.meetings.append(new_meeting)
        print(f"Reunión programada: {new_meeting}")

    def _overlaps(self, m1: Meeting, m2: Meeting) -> bool:
        return m1.start_time < m2.end_time and m2.start_time < m1.end_time

    def list_meetings(self, date: str):
        date_dt = datetime.strptime(date, "%Y-%m-%d").date()
        filtered = [m for m in self.meetings if m.start_time.date() == date_dt]
        for m in sorted(filtered, key=lambda x: x.start_time):
            print(f"[{m.id[:6]}] {m}")

    def cancel_meeting(self, meeting_id: str):
        before = len(self.meetings)
        self.meetings = [m for m in self.meetings if m.id != meeting_id]
        after = len(self.meetings)
        if before == after:
            print("ID no encontrado.")
        else:
            print("Reunión cancelada.")

In [3]:
calendar = MeetingScheduler()

calendar.schedule_meeting("2025-07-17 09:00", 60, "Reunión con cliente")
calendar.schedule_meeting("2025-07-17 10:30", 30, "Planificación interna")

Reunión programada: Reunión con cliente (09:00 - 10:00)
Reunión programada: Planificación interna (10:30 - 11:00)


In [5]:
calendar.list_meetings("2025-07-17")
# → Reunión con cliente (9:00 - 10:00), Planificación interna (10:30 - 11:00)

[57eb5e] Reunión con cliente (09:00 - 10:00)
[90a3e3] Planificación interna (10:30 - 11:00)


In [6]:
calendar.schedule_meeting("2025-07-17 09:30", 30, "Solapada")  
# → Error: se cruza con otra

ValueError: La nueva reunión se cruza con una existente.

# Diseña un sistema de detección de tasa de peticiones (Rate Limiter)

## 💼 Descripción:
Vas a construir un Rate Limiter, es decir, un componente que impide que un usuario (o IP, o servicio) haga demasiadas peticiones en un corto periodo de tiempo.

    - Este patrón es usado en producción por:

    - APIs públicas (OpenAI, Twitter, GitHub)

    - Servicios que protegen de abuso (login, spam, etc.)

## 🎯 Requisitos:
Solo se deben permitir N peticiones por usuario por ventana de tiempo T (por ejemplo, 5 peticiones cada 10 segundos).

    - Rechaza peticiones que excedan ese límite.

    - Opcional: Implementa variantes como sliding window o leaky bucket si quieres ir más lejos.

In [1]:
from collections import deque
from typing import Dict

class RateLimiter:
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.user_requests: Dict[str, deque] = {}

    def allow_request(self, user_id: str, current_time: int) -> bool:
        """
        Registra y decide si la petición puede hacerse en el timestamp actual.
        """
        if user_id not in self.user_requests:
            self.user_requests[user_id] = deque()

        requests = self.user_requests[user_id]

        # Elimina peticiones fuera de la ventana de tiempo
        while requests and current_time - requests[0] >= self.window_seconds:
            requests.popleft()

        if len(requests) < self.max_requests:
            requests.append(current_time)
            return True
        return False

In [4]:
limiter = RateLimiter(max_requests=3, window_seconds=10)

#print(limiter.allow_request("user1", 1))   # True
#print(limiter.allow_request("user1", 2))   # True
#print(limiter.allow_request("user1", 3))   # True
#print(limiter.allow_request("user1", 4))   # False (excede el límite)
print(limiter.allow_request("user1", 12))  # True (ventana reseteada)


True


## Sistema de Clasificación de Productos con Búsqueda y Filtros Eficientes

### 📝 Descripción:
Vas a construir un motor de búsqueda y clasificación de productos (como los usados en Alibaba, JD.com, Pinduoduo), que:

1. Permite registrar productos con:

    - id, nombre, precio, categoría, rating, stock

2. Permite realizar búsquedas filtradas por:

    - Categoría

    - Precio mínimo/máximo

    - Rating mínimo

    - Palabras clave en el nombre

3. Permite ordenar los resultados por:

    - Precio (asc o desc)

    - Rating

    - Relevancia (match en nombre)

In [1]:
from typing import List, Optional

class Product:
    def __init__(self, pid: int, name: str, price: float, category: str, rating: float, stock: int):
        self.pid = pid
        self.name = name
        self.price = price
        self.category = category
        self.rating = rating
        self.stock = stock

    def __repr__(self):
        return f"{self.name} - ${self.price} - ⭐{self.rating}"

class ProductSearchEngine:
    def __init__(self):
        self.products: List[Product] = []

    def add_product(self, pid: int, name: str, price: float, category: str, rating: float, stock: int):
        self.products.append(Product(pid, name, price, category, rating, stock))

    def search(self,
               category: Optional[str] = None,
               min_price: Optional[float] = None,
               max_price: Optional[float] = None,
               min_rating: Optional[float] = None,
               keyword: Optional[str] = None,
               sort_by: Optional[str] = None,
               ascending: bool = False) -> List[Product]:

        results = self.products

        if category:
            results = [p for p in results if p.category.lower() == category.lower()]
        if min_price is not None:
            results = [p for p in results if p.price >= min_price]
        if max_price is not None:
            results = [p for p in results if p.price <= max_price]
        if min_rating is not None:
            results = [p for p in results if p.rating >= min_rating]
        if keyword:
            results = [p for p in results if keyword.lower() in p.name.lower()]

        if sort_by == "price":
            results.sort(key=lambda p: p.price, reverse=not ascending)
        elif sort_by == "rating":
            results.sort(key=lambda p: p.rating, reverse=not ascending)
        elif sort_by == "name":
            results.sort(key=lambda p: p.name)

        return results

In [2]:
store = ProductSearchEngine()

store.add_product(1, "Smartphone Xiaomi Redmi", 799, "Electrónica", 4.5, 10)
store.add_product(2, "Xiaomi Band 7", 199, "Electrónica", 4.8, 5)
store.add_product(3, "Lenovo Laptop IdeaPad", 1200, "Computación", 4.3, 7)

result = store.search(category="Electrónica", min_price=100, max_price=800, sort_by="rating")

# → Devuelve productos 2 y 1 ordenados por rating descendente

In [3]:
result

[Xiaomi Band 7 - $199 - ⭐4.8, Smartphone Xiaomi Redmi - $799 - ⭐4.5]

## Sistema de Reservas de Salas con Conflicto Mínimo (Smart Room Allocator)

### 📝 Descripción:
Vas a construir un sistema que administre reservas de salas en una empresa. Pero con una diferencia clave:
Si no hay sala disponible en el horario solicitado, el sistema debe encontrar la sala con el menor conflicto posible (tiempo libre más cercano) o sugerir una alternativa.

Este ejercicio está inspirado en los sistemas de calendarización de Yandex o Tinkoff.

### 🎯 Requisitos funcionales:
1. Puedes registrar salas (add_room(nombre)).

2. Puedes hacer una reserva (book_room(nombre_sala, fecha, hora_inicio, duración_minutos)).

3. Si no hay disponibilidad exacta:

    - Sugerir la sala con el menor tiempo de conflicto.

    - O decir que no hay forma de agendar hoy.

4. Puedes listar reservas por sala y fecha.

In [4]:
from datetime import datetime, timedelta
from typing import List, Dict

class Reservation:
    def __init__(self, start: datetime, duration: int):
        self.start = start
        self.end = start + timedelta(minutes=duration)

    def overlaps(self, other: 'Reservation') -> bool:
        return self.start < other.end and other.start < self.end

    def __str__(self):
        return f"{self.start.strftime('%H:%M')} - {self.end.strftime('%H:%M')}"

class RoomScheduler:
    def __init__(self):
        self.rooms: Dict[str, List[Reservation]] = {}

    def add_room(self, room_name: str):
        if room_name in self.rooms:
            print("La sala ya existe.")
        else:
            self.rooms[room_name] = []

    def book_room(self, room_name: str, date: str, time: str, duration: int):
        if room_name not in self.rooms:
            print(f"Sala {room_name} no registrada.")
            return

        start = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
        new_reservation = Reservation(start, duration)

        for res in self.rooms[room_name]:
            if res.overlaps(new_reservation):
                print(f"⚠️ Conflicto en {room_name} a las {res}")
                self.suggest_alternative(room_name, date, duration)
                return

        self.rooms[room_name].append(new_reservation)
        print(f"✅ Reserva confirmada en {room_name} de {new_reservation}")

    def suggest_alternative(self, room_name: str, date: str, duration: int):
        print("🔍 Buscando alternativa...")
        for alt_room, reservations in self.rooms.items():
            if alt_room == room_name:
                continue
            start_time = datetime.strptime(f"{date} 08:00", "%Y-%m-%d %H:%M")
            end_time = datetime.strptime(f"{date} 18:00", "%Y-%m-%d %H:%M")

            while start_time + timedelta(minutes=duration) <= end_time:
                trial = Reservation(start_time, duration)
                if all(not r.overlaps(trial) for r in reservations):
                    print(f"💡 Alternativa: reservar {alt_room} a las {start_time.strftime('%H:%M')}")
                    return
                start_time += timedelta(minutes=15)
        print("❌ No hay alternativa disponible hoy.")

    def list_reservations(self, room_name: str, date: str):
        print(f"📋 Reservas para {room_name} en {date}:")
        for r in sorted(self.rooms.get(room_name, []), key=lambda x: x.start):
            if r.start.strftime("%Y-%m-%d") == date:
                print(f" - {r}")

In [5]:
scheduler = RoomScheduler()
scheduler.add_room("Sala A")
scheduler.add_room("Sala B")

scheduler.book_room("Sala A", "2025-07-17", "09:00", 60)
scheduler.book_room("Sala A", "2025-07-17", "10:00", 30)

# Intento hacer reserva que se cruza
scheduler.book_room("Sala A", "2025-07-17", "09:30", 30)
# → Sugerencia: usar Sala B o mover a 10:30

scheduler.list_reservations("Sala A", "2025-07-17")

✅ Reserva confirmada en Sala A de 09:00 - 10:00
✅ Reserva confirmada en Sala A de 10:00 - 10:30
⚠️ Conflicto en Sala A a las 09:00 - 10:00
🔍 Buscando alternativa...
💡 Alternativa: reservar Sala B a las 08:00
📋 Reservas para Sala A en 2025-07-17:
 - 09:00 - 10:00
 - 10:00 - 10:30


## Simulador de Tienda de Café
📝 Descripción:
Vas a crear un programa que simule una tienda de café. El sistema debe:

Mostrar un menú con productos y precios.

Permitir al usuario seleccionar productos y cantidades.

Calcular el total de la compra.

Permitir pagar con diferentes billetes/monedas y dar cambio.

Mostrar un recibo final.


In [1]:
class CoffeeShop:
    def __init__(self):
        self.menu = {
            1: ("Espresso", 3),
            2: ("Cappuccino", 4),
            3: ("Latte", 5)
        }
        self.order = []

    def show_menu(self):
        print("Bienvenido a Café Python ☕")
        for key, (name, price) in self.menu.items():
            print(f"{key}. {name} - ${price}")

    def take_order(self):
        while True:
            choice = int(input("Seleccione un producto (1-3) o 0 para finalizar: "))
            if choice == 0:
                break
            if choice not in self.menu:
                print("Opción inválida.")
                continue
            qty = int(input("Cantidad: "))
            self.order.append((choice, qty))

    def calculate_total(self):
        return sum(self.menu[choice][1] * qty for choice, qty in self.order)

    def process_payment(self, total):
        paid = int(input(f"Total a pagar: ${total}\nIngrese pago: $"))
        if paid < total:
            print("Pago insuficiente.")
            return self.process_payment(total)
        print(f"Cambio: ${paid - total}")

    def print_receipt(self):
        print("\n--- Recibo ---")
        for choice, qty in self.order:
            name, price = self.menu[choice]
            print(f"{name} x{qty} = ${price * qty}")
        print("--------------")

    def run(self):
        self.show_menu()
        self.take_order()
        total = self.calculate_total()
        self.process_payment(total)
        self.print_receipt()
        print("Gracias por su compra.")

# Ejecutar
shop = CoffeeShop()
shop.run()

Bienvenido a Café Python ☕
1. Espresso - $3
2. Cappuccino - $4
3. Latte - $5


ValueError: invalid literal for int() with base 10: ''

Ejercicios básicos:
- Función para saludar: Crea una función que reciba un nombre como parámetro y devuelva un saludo personalizado.
- Función para calcular el área de un círculo: Recibe el radio como parámetro y devuelve el área.
- Función para verificar si un número es par: Recibe un número como parámetro y devuelve True si es par, False en caso contrario.
- Función para convertir grados Celsius a Fahrenheit: Recibe la temperatura en Celsius y devuelve la temperatura en Fahrenheit.
- Función para encontrar el valor máximo de una lista: Recibe una lista de números como parámetro y devuelve el valor máximo. 

Ejercicios con más lógica:
- Función para calcular el factorial de un número: Recibe un número entero positivo y devuelve su factorial. 
- Función para verificar si una cadena es un palíndromo: Recibe una cadena como parámetro y devuelve True si es un palíndromo, False en caso contrario. 
- Función para generar una lista de números primos hasta un límite: Recibe un límite como parámetro y devuelve una lista con los números primos encontrados.
- Función para calcular el mínimo común múltiplo (MCM) de dos números: Recibe dos números como parámetros y devuelve su MCM. 
- Función para calcular el salario de un empleado con horas extras: Recibe las horas trabajadas y la tarifa por hora como parámetros. Si las horas trabajadas superan las 40, calcula las horas extras con una tarifa del 150%. 

Ejercicios con listas y diccionarios: 
- Función para calcular la media de una lista de números: Recibe una lista de números como parámetro y devuelve su media.
- Función para contar la frecuencia de cada elemento en una lista: Recibe una lista como parámetro y devuelve un diccionario con la frecuencia de cada elemento.
- Función para encontrar la intersección de dos listas: Recibe dos listas como parámetros y devuelve una nueva lista con los elementos comunes.
- Función para ordenar una lista de números de forma ascendente: Recibe una lista como parámetro y devuelve la lista ordenada. 

Ejercicios con recursividad:
1. Función recursiva para calcular la secuencia de Fibonacci:
Recibe un número como parámetro y devuelve el término correspondiente de la secuencia de Fibonacci.
2. Función recursiva para buscar un elemento en una lista:
Recibe una lista y un elemento como parámetros y devuelve True si el elemento se encuentra en la lista, False en caso contrario. 

Consejos para resolver los ejercicios:

Comienza con la estructura básica:

Usa la palabra clave def para definir la función, seguida del nombre de la función y los parámetros entre paréntesis.

Define el cuerpo de la función:

Dentro de la función, escribe el código que realizará la tarea deseada.

Utiliza la palabra clave return para devolver el resultado:

Si la función debe devolver un valor, usa return seguido del valor a devolver.

Prueba la función con diferentes entradas:

Llama a la función con diferentes valores para asegurarte de que funciona correctamente.

Divide los problemas complejos en partes más pequeñas:

Si el problema es complejo, puedes dividirlo en varias funciones más pequeñas y luego combinarlas. 

Recuerda que la práctica es la clave para dominar la creación de funciones en Python. ¡Empieza con los ejercicios más sencillos y ve avanzando gradualmente! 

**Función para saludar**: Crea una función que reciba un nombre como parámetro y devuelva un saludo personalizado.

In [ ]:
def greeting(name:str) -> str:
    #name = str(input("Escriba el nombre"))
    saludo = f"Hola {name}"
    return saludo

In [6]:
print(greeting("Randolph"))

Hola Randolph


**Función para calcular el área de un círculo**: Recibe el radio como parámetro y devuelve el área.

In [5]:
def area(radio:float) -> float:
    #radio = float(input("Ingrese el radio"))
    area = radio**2
    return area

In [6]:
print(area(3))

9


**Función para verificar si un número es par**: Recibe un número como parámetro y devuelve True si es par, False en caso contrario.

In [7]:
def paridad(n):
    if n%2 == 0 or n==0:
        print('el numero es par')
    else:
        print('el numero es impar')

In [8]:
x = int(input("Ingrese su numero"))
paridad(x)

el numero es par


**Función para verificar si una cadena es un palíndromo**: Recibe una cadena como parámetro y devuelve True si es un palíndromo, False en caso contrario. 

In [9]:
def es_palindromo(cadena):
  """
  Verifica si una cadena es un palíndromo.

  Args:
    cadena: La cadena a verificar.

  Returns:
    True si la cadena es un palíndromo, False en caso contrario.
  """
  cadena = cadena.lower()
  cadena_invertida = cadena[::-1]
  return cadena == cadena_invertida

In [10]:
cadena1 = "madam"
cadena2 = "hello"

if es_palindromo(cadena1):
  print(f'"{cadena1}" es un palíndromo')
else:
  print(f'"{cadena1}" no es un palíndromo')

if es_palindromo(cadena2):
  print(f'"{cadena2}" es un palíndromo')
else:
  print(f'"{cadena2}" no es un palíndromo')

"madam" es un palíndromo
"hello" no es un palíndromo


**Función para generar una lista de números primos hasta un límite**: Recibe un límite como parámetro y devuelve una lista con los números primos encontrados.

In [1]:
def generar_primos(limite):
    """
    Genera una lista de números primos hasta un límite dado.

    Args:
        limite: El límite superior para buscar números primos.

    Returns:
        Una lista de números primos encontrados.
    """
    primos = []
    for num in range(2, limite + 1):
        es_primo = True
        for i in range(2, int(num**0.5) + 1):
            if num % i == 0:
                es_primo = False
                break
        if es_primo:
            primos.append(num)
    return primos

In [2]:
# Ejemplo de uso:
limite_maximo = 20
lista_primos = generar_primos(limite_maximo)
print(f"Números primos hasta {limite_maximo}: {lista_primos}")

Números primos hasta 20: [2, 3, 5, 7, 11, 13, 17, 19]


## Diseña un sistema de Autocompletado tipo Google Search
## 🧩 Problema
Implementa un sistema de autocompletado donde, dada una lista de frases populares y su frecuencia de búsqueda, puedas:

Registrar las frases ingresadas por el usuario.

Cuando el usuario ingresa un prefijo, el sistema debe sugerir las 3 frases más frecuentes que comiencen con ese prefijo.

Si hay empate en frecuencia, se ordenan alfabéticamente.

## 📥 Input
Lista de frases populares y su frecuencia:
Ejemplo:

python
Copiar
Editar

sentences = ["i love you", "island", "ironman", "i love leetcode"]

times = [5, 3, 2, 2]

El usuario ingresa letra por letra usando el método input(c):

Cada llamada con una letra actualiza las sugerencias.

Si se ingresa el carácter '#', la frase hasta ahora escrita se considera nueva entrada (se guarda o actualiza su frecuencia).

In [1]:
import heapq
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = {}
        self.counts = defaultdict(int)
        self.is_end = False

class AutocompleteSystem:
    def __init__(self, sentences, times):
        self.root = TrieNode()
        self.current_input = ""
        self.curr_node = self.root
        for sentence, time in zip(sentences, times):
            self._add(sentence, time)

    def _add(self, sentence, count):
        node = self.root
        for char in sentence:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
            node.counts[sentence] += count
        node.is_end = True

    def _lookup(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return {}
            node = node.children[char]
        return node.counts

    def input(self, c):
        if c == "#":
            self._add(self.current_input, 1)
            self.current_input = ""
            self.curr_node = self.root
            return []
        self.current_input += c
        counts = self._lookup(self.current_input)
        heap = [(-freq, sentence) for sentence, freq in counts.items()]
        heapq.heapify(heap)
        return [heapq.heappop(heap)[1] for _ in range(min(3, len(heap)))]

In [2]:
ac = AutocompleteSystem(
    ["i love you", "island", "ironman", "i love leetcode"],
    [5, 3, 2, 2]
)

print(ac.input('i'))  # ['i love you', 'i love leetcode', 'island']
#print(ac.input(' '))  # ['i love you', 'i love leetcode']
#print(ac.input('l'))  # ['i love you', 'i love leetcode']
#print(ac.input('o'))  # ['i love you']
#print(ac.input('#'))  # registra "i lo"


['i love you', 'island', 'i love leetcode']


**Función para encontrar el valor máximo de una lista**: Recibe una lista de números como parámetro y devuelve el valor máximo. 

In [1]:
def encontrar_maximo(lista):
  """
  Encuentra el valor máximo en una lista de números.

  Args:
    lista: Una lista de números.

  Returns:
    El valor máximo de la lista.
  """
  return max(lista)


In [2]:
# Ejemplo de uso
numeros = [1, 5, 2, 8, 3]
maximo = encontrar_maximo(numeros)
print(f"El valor máximo es: {maximo}")

El valor máximo es: 8


**Función para calcular el factorial de un número**: Recibe un número entero positivo y devuelve su factorial. 

In [1]:
def factorial(n):
    """
    Cacular el factorial de un numero 
    """
    result = 1
    for i in range(1,n+1):
        result *= i
    return result

In [2]:
x = int(input("Ingrese el numero"))
factorial(x)

720

## Ejercicios de datos simples

## Ejercicio 3
Escribir un programa que pregunte el nombre del usuario en la consola y después de que el usuario lo introduzca muestre por pantalla la cadena ¡Hola <nombre>!, donde <nombre> es el nombre que el usuario haya introducido.

In [1]:
name = str(input("Ingrese su nombre"))
print(f"Hola {name}")

Hola Alan


## Ejercicio 4

Escribir un programa que muestre por pantalla el resultado de la siguiente operación aritmética

<math xmlns="http://www.w3.org/1998/Math/MathML">
  <msup>
    <mrow data-mjx-texclass="INNER">
      <mo data-mjx-texclass="OPEN">(</mo>
      <mfrac>
        <mrow>
          <mn>3</mn>
          <mo>+</mo>
          <mn>2</mn>
        </mrow>
        <mrow>
          <mn>2</mn>
          <mo>&#x22C5;</mo>
          <mn>5</mn>
        </mrow>
      </mfrac>
      <mo data-mjx-texclass="CLOSE">)</mo>
    </mrow>
    <mn>2</mn>
  </msup>
</math>

In [2]:
print(((3+2)/(2*5))**2)

0.25


## Ejercicio 5
Escribir un programa que pregunte al usuario por el número de horas trabajadas y el coste por hora. Después debe mostrar por pantalla la paga que le corresponde.

In [3]:
hours = int(input("Cuantas horas trabajo?"))
cost = int(input("Precio de las"))
print(f"Lo correspondiente a pagar es la siguiente {hours*cost}")

Lo correspondiente a pagar es la siguiente 42000


## Ejercicio 6
Escribir un programa que lea un entero positivo <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>n</mi>
</math>, introducido por el usuario y después muestre en pantalla la suma de todos los enteros desde 1 hasta <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>n</mi>
</math>
. La suma de los <math xmlns="http://www.w3.org/1998/Math/MathML">
  <mi>n</mi>
</math>
 primeros enteros positivos puede ser calculada de la siguiente forma:

 <math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mstyle displaystyle="false" scriptlevel="0">
    <mtext>suma</mtext>
  </mstyle>
  <mo>=</mo>
  <mfrac>
    <mrow>
      <mi>n</mi>
      <mo stretchy="false">(</mo>
      <mi>n</mi>
      <mo>+</mo>
      <mn>1</mn>
      <mo stretchy="false">)</mo>
    </mrow>
    <mn>2</mn>
  </mfrac>
</math>

In [6]:
x = int(input("Ingrese el numero entero"))
if x>=0:
    print((x*(x+1))/2)
else:
    print("No es un numero positivo")

No es un numero positivo


## Ejercicio 7
Escribir un programa que pida al usuario su peso (en kg) y estatura (en metros), calcule el índice de masa corporal y lo almacene en una variable, y muestre por pantalla la frase Tu índice de masa corporal es <imc> donde <imc> es el índice de masa corporal calculado redondeado con dos decimales.

In [9]:
weight = float(input("ingrese su peso en Kg"))
hight = float(input("ingrese su altura en M"))
imc = (weight/hight**2)
print(f"Su indice de masa corporal es de {imc}")

Su indice de masa corporal es de 24.845679012345677


## Ejercicio 8
Escribir un programa que pida al usuario dos números enteros y muestre por pantalla la n entre m da un cociente c y un resto r donde n y m son los números introducidos por el usuario, y c y r son el cociente y el resto de la división entera respectivamente.

In [12]:
n = float(input("ingrese el numero dividendo"))
m = float(input("ingrese el numero divisor"))
c = n/m
r = n%m
print(f"la {n} entre {m} da un cociente {c} y un resto {r} ")

la 4.0 entre 2.0 da un cociente 2.0 y un resto 0.0 


## Ejercicio 9
Escribir un programa que pregunte al usuario una cantidad a invertir, el interés anual y el número de años, y muestre por pantalla el capital obtenido en la inversión.

In [14]:
x = float(input("Cantidad a invertir"))
a = float(input("ingrese los años que desea invertir"))
interes = x*a*(1 - 0.2687)
print(f"Este es el resultado de la capital {interes}")

Este es el resultado de la capital 1462600.0000000002


## Ejercicio 10
Una juguetería tiene mucho éxito en dos de sus productos: payasos y muñecas. Suele hacer venta por correo y la empresa de logística les cobra por peso de cada paquete así que deben calcular el peso de los payasos y muñecas que saldrán en cada paquete a demanda. Cada payaso pesa 112 g y cada muñeca 75 g. Escribir un programa que lea el número de payasos y muñecas vendidos en el último pedido y calcule el peso total del paquete que será enviado.

In [3]:
peso_payaso = 112
peso_muñeca = 75
payasos = int(input("Introduce el número de payasos a enviar: "))
muñecas = int(input("Introduce el número de muñecas a enviar: "))
peso_total = peso_payaso * payasos + peso_muñeca * muñecas
print("El peso total del paquete es " + str(peso_total) + " g")

El peso total del paquete es 449 g


## Ejercicio 11
Imagina que acabas de abrir una nueva cuenta de ahorros que te ofrece el 4% de interés al año. Estos ahorros debido a intereses, que no se cobran hasta finales de año, se te añaden al balance final de tu cuenta de ahorros. Escribir un programa que comience leyendo la cantidad de dinero depositada en la cuenta de ahorros, introducida por el usuario. Después el programa debe calcular y mostrar por pantalla la cantidad de ahorros tras el primer, segundo y tercer años. Redondear cada cantidad a dos decimales.

En algunos casos, las cuentas de ahorros pueden calcular los intereses de forma diaria o mensual. En estos casos, la fórmula del interés simple se modifica para reflejar el período de tiempo más corto:

I = (P * r * t) / 365

Por ejemplo, si una cuenta de ahorros calcula los intereses de forma diaria y tiene una tasa de interés anual del 3%, los intereses generados al cabo de un día serán:

I = (1000 * 0.03 / 365)

I = $0.008

En este caso, los intereses generados al cabo de un año serán de $2.86.

In [ ]:
inversion = float(input("Introduce la inversión inicial: "))
interes = 0.04
balance1 = inversion * (1 + interes)
print("Balance tras el primer año:" + str(round(balance1, 2)))
balance2 = balance1 * (1 + interes)
print("Balance tras el segundo año:" + str(round(balance2, 2)))
balance3 = balance2 * (1 + interes)
print("Balance tras el tercer año:" + str(round(balance3, 2)))

## Ejercicio 12
Una panadería vende barras de pan a 3.49€ cada una. El pan que no es el día tiene un descuento del 60%. Escribir un programa que comience leyendo el número de barras vendidas que no son del día. Después el programa debe mostrar el precio habitual de una barra de pan, el descuento que se le hace por no ser fresca y el coste final total.

In [ ]:
barras = float(input("Ingrese el numero de barras"))
pan = 3.49
descuento = 0.6
coste = pan*barras*(1-descuento)
print(f"El precio de las barras es de {coste} €")
print("El descuento sobre una barra no fresca es " + str(descuento * 100) + "%")
print("El coste final a pagar es " + str(round(coste, 2)) + "€")

## Ejercicios de Cadenas

## Ejercicio 1
Escribir un programa que pregunte el nombre del usuario en la consola y un número entero e imprima por pantalla en líneas distintas el nombre del usuario tantas veces como el número introducido.

In [1]:
nombre = input("¿Cómo te llamas? ")
n = input("Introduce un número entero: ")
print((nombre + "\n") * int(n))

Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph
Randolph



## Ejercicio 2
Escribir un programa que pregunte el nombre completo del usuario en la consola y después muestre por pantalla el nombre completo del usuario tres veces, una con todas las letras minúsculas, otra con todas las letras mayúsculas y otra solo con la primera letra del nombre y de los apellidos en mayúscula. El usuario puede introducir su nombre combinando mayúsculas y minúsculas como quiera.

In [6]:
complet_name = str(input("Introduzca su nombre completo"))
print(complet_name.upper() + '\n' + complet_name.lower() + '\n' + complet_name.title())

RANDOLPH RAFAEL PERALTA MONTERROZA
randolph rafael peralta monterroza
Randolph Rafael Peralta Monterroza


## Ejercicio 3
Escribir un programa que pregunte el nombre del usuario en la consola y después de que el usuario lo introduzca muestre por pantalla NOMBRE tiene n letras, donde NOMBRE es el nombre de usuario en mayúsculas y n es el número de letras que tienen el nombre.

In [1]:
N = str(input("Introduzca su nombre"))
n = len(N)
print(f"el nombre de {N.upper()} tiene {n} letras")

el nombre de RANDOLPH tiene 8 letras


## Ejercicio 4
Los teléfonos de una empresa tienen el siguiente formato prefijo-número-extension donde el prefijo es el código del país +34, y la extensión tiene dos dígitos (por ejemplo +34-913724710-56). Escribir un programa que pregunte por un número de teléfono con este formato y muestre por pantalla el número de teléfono sin el prefijo y la extensión.

In [4]:
tel = input("Introduce un número de teléfono con el formato +xx-xxxxxxxxx-xx: ")
print('El número de teléfono es ', tel[4:-3])

El número de teléfono es  913724710


## Ejercicio 5
Escribir un programa que pida al usuario que introduzca una frase en la consola y muestre por pantalla la frase invertida.

In [5]:
name = str(input("Ingrese su nombre"))
name_inverted = name[::-1]
print(name_inverted)

hplodnaR


## Ejercicio 6
Escribir un programa que pida al usuario que introduzca una frase en la consola y una vocal, y después muestre por pantalla la misma frase pero con la vocal introducida en mayúscula.

In [7]:
frase = str(input("Introduzca su frase"))
vocal = str(input("Introduzca una vocal"))
print(frase + " " + vocal.upper())

Fuerza A


## Ejercicio 7
Escribir un programa que pregunte el correo electrónico del usuario en la consola y muestre por pantalla otro correo electrónico con el mismo nombre (la parte delante de la arroba @) pero con dominio ceu.es.

In [2]:
email = input("Introduce tu correo electrónico: ")
print(email[:email.find('@')] + '@ceu.es')

asalbe@ceu.es


## Ejercicio 8
Escribir un programa que pregunte por consola el precio de un producto en euros con dos decimales y muestre por pantalla el número de euros y el número de céntimos del precio introducido.

In [14]:
precio = str(input("Introduce el precio del producto con dos decimales:  "))
print(f"Se tiene {precio[:precio.find(".")]} euros, donde  hay {precio[precio.find(".")+1:]} centimos")

Se tiene 123 euros, donde  hay 654 centimos


## Ejercicio 9
Escribir un programa que pregunte al usuario la fecha de su nacimiento en formato dd/mm/aaaa y muestra por pantalla, el día, el mes y el año. Adaptar el programa anterior para que también funcione cuando el día o el mes se introduzcan con un solo carácter.

In [14]:
fecha = str(input("ingrese su fecha de nacimiento"))
fecha = fecha.split("/")
dia, mes, año = fecha
print(f"Se tiene {dia} del mes {mes} del año {año}")

Se tiene 12 del mes 8 del año 2021


## Ejercicio 10
Escribir un programa que pregunte por consola por los productos de una cesta de la compra, separados por comas, y muestre por pantalla cada uno de los productos en una línea distinta.

In [20]:
productos = str(input("Introduzca los productos de su cesta de compras"))
productos = productos.split(",")
for producto in productos:
    print(producto)

manzana
 pera
 maracuya
 naranja


## Ejercicio 11
Escribir un programa que pregunte el nombre el un producto, su precio y un número de unidades y muestre por pantalla una cadena con el nombre del producto seguido de su precio unitario con 6 dígitos enteros y 2 decimales, el número de unidades con tres dígitos y el coste total con 8 dígitos enteros y 2 decimales.

In [ ]:
producto = str(input("Cual es su producto"))
precio = float(input("Cual es su precio"))
unidad = int(input("Cuantos desea"))
print('{producto}: {unidad:3d} unidad x {precio:09.2f}€ = {total:011.2f}€'.format(producto = producto, unidad = unidad, precio = precio, total = unidad * precio))

Manzana:   1 unidad x 000090.00€ = 00000090.00€


## Ejercicios de Condicionales

## Ejercicio 1
Escribir un programa que pregunte al usuario su edad y muestre por pantalla si es mayor de edad o no.

In [2]:
edad = int(input("ingrse su edad"))
if edad >= 18:
    print("Eres mayor de edad")
else:
    print("Eres menor de edad")

Eres mayor de edad


## Ejercicio 2
Escribir un programa que almacene la cadena de caracteres contraseña en una variable, pregunte al usuario por la contraseña e imprima por pantalla si la contraseña introducida por el usuario coincide con la guardada en la variable sin tener en cuenta mayúsculas y minúsculas.

In [3]:
key = "1234qwer"
password = str(input("introduzca la contraseña del usuario: "))
if password.lower() == key:
    print("Contraseña correcta")
else:
    print("Contraseña incorrecta")

Contraseña correcta


## Ejercicio 3
Escribir un programa que pida al usuario dos números y muestre por pantalla su división. Si el divisor es cero el programa debe mostrar un error.

In [7]:
dividendo = float(input("Ingrese su numero: "))
divisor = float(input("Ingrese el numero divisor: "))
if divisor != 0:
    print(f"La divison es {dividendo/divisor}")
else:
    print("El divisor es cero, ingrese nuevamnte la division")

La divison es 2.0


# Ejercicio 4
Escribir un programa que pida al usuario un número entero y muestre por pantalla si es par o impar.

In [16]:
numero = int(input("Ingrese su numero"))
if numero % 2 == 0:
    print(f"El numero {numero} par")
else:
    print(f"El numero {numero} impar")

El numero -9 impar


## Ejercicio 5
Para tributar un determinado impuesto se debe ser mayor de 16 años y tener unos ingresos iguales o superiores a 1000 € mensuales. Escribir un programa que pregunte al usuario su edad y sus ingresos mensuales y muestre por pantalla si el usuario tiene que tributar o no.

In [17]:
edad = int(input("ingrese su edad"))
ingresos = float(input("ingrese sus ingresos mensuales"))
if edad >= 16 and ingresos >= 1000:
    print("Debe tributar")
else:
    print("No debe tributar")

No debe tributar


## Ejercicio 6
Los alumnos de un curso se han dividido en dos grupos A y B de acuerdo al sexo y el nombre. El grupo A esta formado por las mujeres con un nombre anterior a la M y los hombres con un nombre posterior a la N y el grupo B por el resto. Escribir un programa que pregunte al usuario su nombre y sexo, y muestre por pantalla el grupo que le corresponde.

In [ ]:
name = input("¿Cómo te llamas? ")
gender = input("¿Cuál es tu sexo (M o H)? ")
if gender == "M":
    if name.lower() < "m":
        group = "A"
    else:
        group = "B"
else:
    if name.lower() > "n":
        group = "A"
    else:
        group = "B"
print("Tu grupo es " + group)

## Ejercicio 7
Los tramos impositivos para la declaración de la renta en un determinado país son los siguientes:

```
Renta	                Tipo impositivo
Menos de 10000€	             5%
Entre 10000€ y 20000€	    15%
Entre 20000€ y 35000€	    20%
Entre 35000€ y 60000€	    30%
Más de 60000€	            45%
```

Escribir un programa que pregunte al usuario su renta anual y muestre por pantalla el tipo impositivo que le corresponde.

In [ ]:
renta = float(input("ingrese su renta"))
if renta < 10000:
    tipo = 5
elif renta < 20000:
    tipo = 15
elif renta < 35000:
    tipo = 20
elif renta < 60000:
    tipo = 30
else:
    tipo = 45
# Mostrar por pantalla el producto de la renta por el tipo impositivo
print("Tienes que pagar ", renta * tipo / 100, "€", sep='')

14814.81


## Ejercicio 8
En una determinada empresa, sus empleados son evaluados al final de cada año. Los puntos que pueden obtener en la evaluación comienzan en 0.0 y pueden ir aumentando, traduciéndose en mejores beneficios. Los puntos que pueden conseguir los empleados pueden ser 0.0, 0.4, 0.6 o más, pero no valores intermedios entre las cifras mencionadas. A continuación se muestra una tabla con los niveles correspondientes a cada puntuación. La cantidad de dinero conseguida en cada nivel es de 2.400€ multiplicada por la puntuación del nivel.

```
Nivel	        Puntuación
Inaceptable	    0.0
Aceptable	    0.4
Meritorio	    0.6 o más
```

Escribir un programa que lea la puntuación del usuario e indique su nivel de rendimiento, así como la cantidad de dinero que recibirá el usuario.

In [5]:
bonificacion = 2400
inaceptable = 0
aceptable = 0.4
meritorio = 0.6
puntos = float(input("Introduce tu puntuación: "))
# Clasifiación por niveles de rendimiento
if puntos == inaceptable:
    nivel = "Inaceptable"
elif puntos == aceptable:
    nivel = "Aceptable"
elif puntos >= 0.6:
    nivel = "Meritorio"
else:
    nivel = ""
# Mostrar nivel de rendimiento
if nivel == "":
    print("Esta puntuación no es válida")
else:
    print("Tu nivel de rendimiento es %s" % nivel)
    print("Te corresponde cobrar %.2f€" % (puntos * bonificacion))

Tu nivel de rendimiento es Meritorio
Te corresponde cobrar 2400.00€


## Ejercicio 9
Escribir un programa para una empresa que tiene salas de juegos para todas las edades y quiere calcular de forma automática el precio que debe cobrar a sus clientes por entrar. El programa debe preguntar al usuario la edad del cliente y mostrar el precio de la entrada. Si el cliente es menor de 4 años puede entrar gratis, si tiene entre 4 y 18 años debe pagar 5€ y si es mayor de 18 años, 10€.

## Ejercicios de diccionarios

## Ejercicio 1
Escribir un programa que guarde en una variable el diccionario {'Euro':'€', 'Dollar':'$', 'Yen':'¥'}, pregunte al usuario por una divisa y muestre su símbolo o un mensaje de aviso si la divisa no está en el diccionario.

In [3]:
monedas = {'Euro':'€', 'Dollar':'$', 'Yen':'¥'}
moneda = input("Introduce una divisa: ")
print(monedas.get(moneda.title(), "La divisa no está."))

€


## Ejercicio 2
Escribir un programa que pregunte al usuario su nombre, edad, dirección y teléfono y lo guarde en un diccionario. Después debe mostrar por pantalla el mensaje <nombre> tiene <edad> años, vive en <dirección> y su número de teléfono es <teléfono>.

In [ ]:
nombre = str(input("Cual es tu nombre?"))
edad = int(input("Cual es tu edad?"))
direccion = str(input("Cual es tu direccion?"))
telefono = int(input("Cual es tu numero de telefono o celular"))
persona = {'nombre': nombre, 'edad': edad, 'direccion': direccion, 'telefono':telefono}
print(f"Te llamas {persona['nombre']}, tu edad es de {persona['edad']} vives en esta direccion {persona['direccion']} y tu telefono es {persona['telefono']}")

Te llamas Randolph y tu edad es de 29


## Ejercicio 3
Escribir un programa que guarde en un diccionario los precios de las frutas de la tabla, pregunte al usuario por una fruta, un número de kilos y muestre por pantalla el precio de ese número de kilos de fruta. Si la fruta no está en el diccionario debe mostrar un mensaje informando de ello.

```
Fruta	Precio
Plátano	 1.35
Manzana	 0.80
Pera	 0.85
Naranja	 0.70
```

In [3]:
fruta = str(input("Ingrese la fruta"))
cantidad = int(input("Cuantos kilos desea llevar"))

frutas = {'Platano':1.35, 'Manzana':0.80, 'Pera':0.85, 'Naranja':0.70}
if fruta in frutas:
    precio = frutas[fruta]
    total = cantidad * precio
    print(f"El precio de {cantidad} kilos de {fruta} es: ${total:.2f}")
else:
    print("la fruta no esta en la lista")

El precio de 2 kilos de Manzana es: $1.60


## Ejercicio 4
Escribir un programa que pregunte una fecha en formato dd/mm/aaaa y muestre por pantalla la misma fecha en formato dd de <mes> de aaaa donde <mes> es el nombre del mes.

In [7]:
meses = {
    '01': 'enero',
    '02': 'febrero',
    '03': 'marzo',
    '04': 'abril',
    '05': 'mayo',
    '06': 'junio',
    '07': 'julio',
    '08': 'agosto',
    '09': 'septiembre',
    '10': 'octubre',
    '11': 'noviembre',
    '12': 'diciembre'
}

fecha = input("Ingrese la fecha en formato dd/mm/aaaa: ")
partes = fecha.split('/')

if len(partes) == 3 and partes[1] in meses:
    dia, mes, año = partes
    print(f"{dia} de {meses[mes]} de {año}")
else:
    print("Formato de fecha inválido o mes no reconocido.")

01 de enero de 2001


## Ejercicio 5
Escribir un programa que almacene el diccionario con los créditos de las asignaturas de un curso {'Matemáticas': 6, 'Física': 4, 'Química': 5} y después muestre por pantalla los créditos de cada asignatura en el formato <asignatura> tiene <créditos> créditos, donde <asignatura> es cada una de las asignaturas del curso, y <créditos> son sus créditos. Al final debe mostrar también el número total de créditos del curso.

In [5]:
curso = {'Matemáticas': 6, 'Física': 4, 'Química': 5}
total_creditos = 0
for asignatura, creditos in curso.items():
    print(asignatura, 'tiene', creditos, 'créditos')
    total_creditos += creditos
print('Número total de créditos del curso: ', total_creditos)

Matemáticas tiene 6 créditos
Física tiene 4 créditos
Química tiene 5 créditos
Número total de créditos del curso:  15


## Ejercicio 6
Escribir un programa que cree un diccionario vacío y lo vaya llenado con información sobre una persona (por ejemplo nombre, edad, sexo, teléfono, correo electrónico, etc.) que se le pida al usuario. Cada vez que se añada un nuevo dato debe imprimirse el contenido del diccionario.

In [2]:
persona= {}
continuar = True
while continuar:
    clave = input("Ingrese el dato que desea guardar")
    valor = input(clave + ': ')
    persona[clave] = valor
    print(persona)
    conitnuar = input("Deseas continuar Si / No?")
    if continuar == 'Si':
        continue
    else:
        break


{'Nombre': 'Randolph'}


## Ejercicio 7
Escribir un programa que cree un diccionario simulando una cesta de la compra. El programa debe preguntar el artículo y su precio y añadir el par al diccionario, hasta que el usuario decida terminar. Después se debe mostrar por pantalla la lista de la compra y el coste total, con el siguiente formato

```
Lista de la compra	

Artículo 1	Precio
Artículo 2	Precio
Artículo 3	Precio
…	…  ... ... ...
Total	    Coste
```

In [1]:
cesta = {}
continuar = True
while continuar:
    item = input('Introduce un artículo: ')
    precio = float(input('Introduce el precio de ' + item + ': '))
    cesta[item] = precio
    continuar = input('¿Quieres añadir artículos a la lista (Si/No)? ') == "Si"
coste = 0
print('Lista de la compra')
for item, precio in cesta.items():
    print(item, '\t', precio)
    coste += precio
print('Coste total: ', coste)

Lista de la compra
Mesa 	 100.0
Silla 	 200.0
Coste total:  300.0
